In [1]:
import os
import cv2
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [12]:
def load_images_from_folder(base_folder):
    images = []
    labels = []
    for label in ["plastic", "no-plastic"]:
        folder = os.path.join(base_folder, label)
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  
            if gray is not None:
               

                # 🔍 Aplicar Canny para detecção de bordas
                edges = cv2.Canny(gray, threshold1=100, threshold2=200)

                # 💡 (Opcional) Dilatar as bordas para garantir contornos fechados
                kernel = np.ones((3, 3), np.uint8)
                edges_dilated = cv2.dilate(edges, kernel, iterations=1)

                # 🧠 Encontrar contornos
                contours, _ = cv2.findContours(edges_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cropped = []
                if contours:
                    # Selecionar o maior contorno
                    cnt = max(contours, key=cv2.contourArea)

                    # 🎯 Criar máscara do contorno
                    mask = np.zeros_like(gray)
                    cv2.drawContours(mask, [cnt], -1, 255, -1)  # Preencher o contorno

                    # 🩹 Aplicar a máscara na imagem original
                    result = cv2.bitwise_and(gray, gray, mask=mask)

                    # ✂️ Crop na região do objeto
                    x, y, w, h = cv2.boundingRect(cnt)
                    cropped = result[y:y+h, x:x+w]
                else:
                    cropped = gray


                cropped = cv2.resize(cropped, (256,256))
                cropped = cropped / 255.0 
                gray_centered = cropped - np.mean(cropped, axis=0)
                pca = PCA(n_components=150)
                transformed = pca.fit_transform(gray_centered)
                reconstructed = pca.inverse_transform(transformed)
                reconstructed += np.mean(cropped, axis=0)

                img_flattened = reconstructed.flatten()  
                images.append(img_flattened)
                labels.append(1 if label == "plastic" else 0)  
    return np.array(images), np.array(labels)

In [13]:

X_train, y_train = load_images_from_folder("SOUVIK/train")
X_test, y_test = load_images_from_folder("SOUVIK/test")

In [14]:
print(f"Treino: {X_train.shape}, Rótulos: {y_train.shape}")
print(f"Teste: {X_test.shape}, Rótulos: {y_test.shape}")

Treino: (1720, 65536), Rótulos: (1720,)
Teste: (430, 65536), Rótulos: (430,)


## Teste com SVM

In [21]:
from sklearn.svm import SVC
svc = SVC(kernel='poly')
svc.fit(X_train, y_train)

SVC(kernel='poly')

### SVC com kernel rbf

In [16]:
y_pred_svc = svc.predict(X_test)
class_rep = classification_report(y_test, y_pred_svc)
print(f"Classification Report")
print(class_rep)

cm = confusion_matrix(y_test, y_pred_svc)
print(f"Confusion Matrix")
print(cm)
print(f"TN: {cm[0][0]} | FN: {cm[0][1]} | FP: {cm[1][0]} | TP: {cm[1][1]}")

Classification Report
              precision    recall  f1-score   support

           0       0.62      0.58      0.60       220
           1       0.59      0.63      0.61       210

    accuracy                           0.60       430
   macro avg       0.61      0.61      0.60       430
weighted avg       0.61      0.60      0.60       430

Confusion Matrix
[[127  93]
 [ 77 133]]
TN: 127 | FN: 93 | FP: 77 | TP: 133


## KNN

In [23]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [24]:
y_pred_svc = knn.predict(X_test)
class_rep = classification_report(y_test, y_pred_svc)
print(f"Classification Report")
print(class_rep)

cm = confusion_matrix(y_test, y_pred_svc)
print(f"Confusion Matrix")
print(cm)
print(f"TN: {cm[0][0]} | FN: {cm[0][1]} | FP: {cm[1][0]} | TP: {cm[1][1]}")

Classification Report
              precision    recall  f1-score   support

           0       0.58      0.64      0.61       220
           1       0.58      0.52      0.55       210

    accuracy                           0.58       430
   macro avg       0.58      0.58      0.58       430
weighted avg       0.58      0.58      0.58       430

Confusion Matrix
[[140  80]
 [101 109]]
TN: 140 | FN: 80 | FP: 101 | TP: 109


### SVC com kernel poly

In [22]:
y_pred_svc = svc.predict(X_test)
class_rep = classification_report(y_test, y_pred_svc)
print(f"Classification Report")
print(class_rep)

cm = confusion_matrix(y_test, y_pred_svc)
print(f"Confusion Matrix")
print(cm)
print(f"TN: {cm[0][0]} | FN: {cm[0][1]} | FP: {cm[1][0]} | TP: {cm[1][1]}")

Classification Report
              precision    recall  f1-score   support

           0       0.48      0.21      0.30       220
           1       0.48      0.76      0.59       210

    accuracy                           0.48       430
   macro avg       0.48      0.49      0.44       430
weighted avg       0.48      0.48      0.44       430

Confusion Matrix
[[ 47 173]
 [ 51 159]]
TN: 47 | FN: 173 | FP: 51 | TP: 159


### SVC com kernel linear

In [10]:
y_pred_svc = svc.predict(X_test)
class_rep = classification_report(y_test, y_pred_svc)
print(f"Classification Report")
print(class_rep)

cm = confusion_matrix(y_test, y_pred_svc)
print(f"Confusion Matrix")
print(cm)
print(f"TN: {cm[0][0]} | FN: {cm[0][1]} | FP: {cm[1][0]} | TP: {cm[1][1]}")

Classification Report
              precision    recall  f1-score   support

           0       0.57      0.64      0.60       220
           1       0.57      0.49      0.53       210

    accuracy                           0.57       430
   macro avg       0.57      0.57      0.56       430
weighted avg       0.57      0.57      0.56       430

Confusion Matrix
[[141  79]
 [107 103]]
TN: 141 | FN: 79 | FP: 107 | TP: 103


### SVC com kernel sigmoid

In [13]:
y_pred_svc = svc.predict(X_test)
class_rep = classification_report(y_test, y_pred_svc)
print(f"Classification Report")
print(class_rep)

cm = confusion_matrix(y_test, y_pred_svc)
print(f"Confusion Matrix")
print(cm)
print(f"TN: {cm[0][0]} | FN: {cm[0][1]} | FP: {cm[1][0]} | TP: {cm[1][1]}")

Classification Report
              precision    recall  f1-score   support

           0       0.48      0.46      0.47       220
           1       0.46      0.49      0.47       210

    accuracy                           0.47       430
   macro avg       0.47      0.47      0.47       430
weighted avg       0.47      0.47      0.47       430

Confusion Matrix
[[101 119]
 [108 102]]
TN: 101 | FN: 119 | FP: 108 | TP: 102


## Teste com Kmeans

USANDO ÁRVORE

In [25]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_train)

KMeans(n_clusters=2, random_state=42)

In [26]:
y_pred_kmeans = kmeans.fit_predict(X_test)

In [27]:
from sklearn.metrics import adjusted_rand_score

print("Adjusted Rand Index:", adjusted_rand_score(y_test, y_pred_kmeans))

Adjusted Rand Index: 0.039730540974146994


In [28]:
import numpy as np
from scipy.stats import mode


labels_map = np.zeros_like(y_pred_kmeans)

for cluster in range(2):
    
    mask = (y_pred_kmeans == cluster)
  
    labels_map[mask] = mode(y_test[mask], keepdims=True).mode[0]

In [29]:
class_rep = classification_report(y_test, y_pred_kmeans)
print(f"Classification Report")
print(class_rep)

cm = confusion_matrix(y_test, y_pred_kmeans)
print(f"Confusion Matrix")
print(cm)
print(f"TN: {cm[0][0]} | FN: {cm[0][1]} | FP: {cm[1][0]} | TP: {cm[1][1]}")

Classification Report
              precision    recall  f1-score   support

           0       0.38      0.29      0.33       220
           1       0.41      0.51      0.45       210

    accuracy                           0.40       430
   macro avg       0.40      0.40      0.39       430
weighted avg       0.39      0.40      0.39       430

Confusion Matrix
[[ 64 156]
 [103 107]]
TN: 64 | FN: 156 | FP: 103 | TP: 107


## Testando árvore de decisão

## Testando o Ensenble learning

Modelos escolhidos

In [16]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [21]:
tree = DecisionTreeClassifier(max_depth=2)

In [22]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=2)

In [24]:
y_pred = tree.predict(X_test)
print("Acurácia:", classification_report(y_test, y_pred))

Acurácia:               precision    recall  f1-score   support

           0       0.59      0.47      0.53       220
           1       0.55      0.66      0.60       210

    accuracy                           0.57       430
   macro avg       0.57      0.57      0.56       430
weighted avg       0.57      0.57      0.56       430



INDIVIDUAL TEST. CHANGE THE PATH TO INPUT IMAGES

0 IS NO_PLASTIC AND 1 IS PLASTIC

In [16]:
import pickle

# Salve o modelo em um arquivo
filename = 'knn_model.pkl'
pickle.dump(knn, open(filename, 'wb'))

print(f"Modelo KNN exportado para {filename}")

Modelo KNN exportado para knn_model.pkl
